In [1]:
import sys
import pandas as pd
import numpy as np
import nasdaqdatalink as ndl
import os
import psycopg2 as psg



In [2]:
ndl.ApiConfig.api_key = os.getenv('NASDAQ_DATA_LINK_API_KEY')
# try:
#fetching region Table
region_table = pd.DataFrame(ndl.get_table('ZILLOW/REGIONS', paginate = True))



# except Exception as e:
#     print(f"Error: {e}")




In [3]:

# region_table['region'].astype(str)
# region_table['region_type'].astype(str)

#filtering df for only CA regions
california_regionsdf = region_table[region_table['region'].str.contains('CA', na = False)]

#print(california_regionsdf)

#List of available regions
region_types = list(california_regionsdf.drop_duplicates(subset = ['region_type'])['region_type'])

#remove these as i do not want to track info for these region types
region_types.remove('neigh')
region_types.remove('metro')


print(region_types)



['city', 'zip', 'county']


In [4]:
#creating DataFrames for each city, county, zip#

#creating df for city
#creating copy of the data frame that only contains city types
full_city_df = california_regionsdf[california_regionsdf['region_type'].str.contains(region_types[0])].copy()

#print(full_city_df)

#adding the state and city section from the region info
full_city_df[['city','state','_']] = full_city_df['region'].str.split(';', n = 2, expand = True)

#city data frame that only contains needed data for user
city_df = full_city_df[['region_id','region_type', 'city', 'state']]

#print(city_df)



In [5]:
#creating data frame for zip

#create copy df of regions that are only zip info
full_zip_df = california_regionsdf[california_regionsdf['region_type'].str.contains(region_types[1])].copy()

#add zipcode and state columns to df
full_zip_df[['zipcode', 'state', '_']] = full_zip_df['region'].str.split(';', n = 2, expand = True)

#df to be pushed to db
zip_df = full_zip_df[['region_id', 'region_type', 'zipcode', 'state']]

#print(zip_df)

In [59]:
#creating df for county

full_county_df = california_regionsdf[california_regionsdf['region_type'].str.contains(region_types[2])].copy()

#add county and state columns to df
full_county_df[['county', 'state', '_']] = full_county_df['region'].str.split(';', n = 2, expand = True)

#df to be pushed to db
county_df = full_county_df[['region_id', 'region_type', 'county', 'state']]
#print(county_df)


In [58]:
indicator_table_full = pd.DataFrame(ndl.get_table('ZILLOW/INDICATORS'))

raw_home_indicators = indicator_table_full[indicator_table_full['indicator'].str.contains('All Homes') & indicator_table_full['indicator'].str.contains('Raw')].copy()

raw_monthly_home_indicators = raw_home_indicators[raw_home_indicators['indicator'].str.contains('Monthly')].copy()

raw_monthly_home_indicators = pd.concat([raw_monthly_home_indicators, indicator_table_full.iloc[[3]]], ignore_index=True)

#print(raw_monthly_home_indicators)

In [41]:

la_region = county_df[county_df['county'].str.contains('Los Angeles')]['region_id']

la_region_id = la_region.iloc[0]

#print(la_region_id)
home_data_table = pd.DataFrame(ndl.get_table('ZILLOW/DATA', indicator_id = 'ISAM', region_id = la_region_id, paginate= True))

home_data_values = home_data_table['value'].round(2)

print(home_data_values)

Series([], Name: value, dtype: object)
